# Análise e Limpeza de Dados: Dataset fakeTelegram.BR_2022

**Disciplina:** CKP9011 - Introdução à Ciência de Dados / CK0223 - Mineração de Dados
**Instituição:** Universidade Federal do Ceará (UFC)
**Atividade:** Lista de Exercícios 1 - Tratamento de Dados

## Introdução

Este notebook documenta o processo completo de tratamento e limpeza de dados aplicado ao dataset `fakeTelegram.BR_2022.csv`, conforme solicitado na Lista de Exercícios 1. O objetivo é transformar os dados brutos em um conjunto de dados consistente, limpo e enriquecido, pronto para futuras análises.

As etapas realizadas seguem as tarefas propostas na lista, abrangendo desde a identificação de valores nulos até a verificação de inconsistências lógicas entre atributos.

In [2]:
import pandas as pd
import numpy as np

file = pd.read_csv('../data/raw/fakeTelegram.BR_2022.csv')

## 2. Análise de Dados Faltantes

A primeira etapa do tratamento de dados é a identificação de valores ausentes (`NaN`), que podem impactar a qualidade de qualquer análise subsequente.

### 2.1. Localização e Visualização dos Valores Nulos

Para entender a distribuição dos dados faltantes, criei funções para:
1.  **Localizar as coordenadas exatas** de cada célula nula de forma eficiente, utilizando `numpy.where`.
2.  **Estruturar essas coordenadas** em um DataFrame para facilitar a consulta e a visualização.

In [4]:
def nulls_cells(df):
    coords = np.where(df.isnull())
    coords_list = list(zip(coords[0], coords[1]))
    return coords_list

def visualize_nulls(coords_list):
    nulls_index = []
    
    for linha, coluna_idx in coords_list:
        column_name = file.columns[coluna_idx]
        nulls_index.append({
            'LINHA': linha,
            'COLUNA': column_name
        })

    if not nulls_index:
        return print('Não há valores nulos no df.')
    return pd.DataFrame(nulls_index)

df_nulls = visualize_nulls(nulls_cells(file))
df_nulls.head(20)

,LINHA,COLUNA
0,0,media
1,0,media_type
2,0,media_url
3,0,score_misinformation
4,0,media_name
5,0,media_md5
6,1,id_member_anonymous
7,1,media
8,1,media_type
9,1,media_url


### 2.2. Quantificação dos Valores Nulos

Após a localização, quantifiquei os dados faltantes de duas formas:
* **Contagem Total:** Um número geral de todas as células vazias no dataset (cumprindo a tarefa c).
* **Contagem por Coluna:** Uma análise agregada que mostra quais features (colunas) são as mais afetadas pela ausência de dados (cumprindo a tarefa d).

In [5]:
def total_nulls(df):
    return df.isnull().sum().sum()

def nulls_by_column(df):
    counter = df.isnull().sum()
    return counter.sort_values(ascending=False)

print(f"Total de células com valores nulos: {total_nulls(file)}")
print("\nContagem de nulos por coluna:")
print(nulls_by_column(file))

Total de células com valores nulos: 2544186

Contagem de nulos por coluna:
media_name                528599
media_url                 400141
score_misinformation      390348
id_member_anonymous       323341
media_md5                 224981
media_type                224981
media                     224981
score_sentiment           113429
text_content_anonymous    113385
id_group_anonymous             0
date_message                   0
dataset_info_id                0
trava_zap                      0
has_media_url                  0
has_media                      0
id_message                     0
date_system                    0
messenger                      0
message_type                   0
dtype: int64


## 3. Verificação de Linhas Duplicadas

Registros duplicados podem introduzir vieses e inflar métricas. Nesta etapa (tarefa e), identifiquei se existem linhas **inteiramente idênticas** no DataFrame. Utilizando `df.duplicated(keep=False)` para marcar todas as ocorrências de uma duplicata.

In [6]:
def duplicated_rows(df):
        if df.duplicated(keep=False).sum() == 0:
            return print('Não há linhas duplicadas no df.')
        else:
            return df[df.duplicated(keep=False)]

duplicated_rows(file)

Não há linhas duplicadas no df.


## 4. Validação de Domínio dos Dados

Esta etapa (tarefa f) busca por células que, embora não nulas, contêm dados que não correspondem ao tipo ou categoria esperada. A validação foi feita da seguinte forma:
-   **Para colunas numéricas:** Utilizei `pd.to_numeric` com `errors='coerce'`, que transforma qualquer valor não-numérico em `NaN`.
-   **Para colunas categóricas:** Usei `isin()` para verificar se os valores pertencem a uma lista pré-definida de categorias válidas.

In [19]:
erros_de_dominio = []

numeric_columns_waited= ['score_misinformation', 'caracteres', 'words', 'sharings', 'sentimental']

for coluna in numeric_columns_waited:
    coluna_convertida = pd.to_numeric(file[coluna], errors='coerce')
    
    erro = coluna_convertida.isnull() & file[coluna].notnull()
    if erro.any():
        for indice, valor in file[erro][coluna].items():
            erros_de_dominio.append({
                'Linha': indice,
                'Coluna': coluna,
                'Valor_Invalido': valor,
                'Tipo_Esperado': 'Numérico'
            })

msg_types_waited = ['Texto', 'Imagem']
erro = ~file['message_type'].isin(msg_types_waited) & file['message_type'].notnull()

if erro.any():
    for indice, valor in file[erro]['message_type'].items():
        erros_de_dominio.append({
            'Linha': indice,
            'Coluna': 'message_type',
            'Valor_Invalido': valor,
            'Tipo_Esperado': f'Um de {msg_types_waited}'
        })

if erros_de_dominio:
    df_erros = pd.DataFrame(erros_de_dominio)
    print(f"Foram encontrados {len(df_erros)} erros de domínio:")
    print(df_erros.head(20))
else:
    print("Nenhum erro de domínio foi encontrado com base nas regras definidas.")

Foram encontrados 128998 erros de domínio:
    Linha        Coluna Valor_Invalido              Tipo_Esperado
0      57  message_type          Video  Um de ['Texto', 'Imagem']
1      61  message_type            Url  Um de ['Texto', 'Imagem']
2      62  message_type          Video  Um de ['Texto', 'Imagem']
3      81  message_type            Url  Um de ['Texto', 'Imagem']
4      83  message_type          Video  Um de ['Texto', 'Imagem']
5      96  message_type          Video  Um de ['Texto', 'Imagem']
6      99  message_type            Url  Um de ['Texto', 'Imagem']
7     118  message_type            Url  Um de ['Texto', 'Imagem']
8     133  message_type            Url  Um de ['Texto', 'Imagem']
9     143  message_type            Url  Um de ['Texto', 'Imagem']
10    146  message_type            Url  Um de ['Texto', 'Imagem']
11    150  message_type            Url  Um de ['Texto', 'Imagem']
12    179  message_type            Url  Um de ['Texto', 'Imagem']
13    180  message_type          

## 5. Engenharia de Atributos

Para enriquecer o dataset e facilitar análises futuras (tarefas g a k), criarei um conjunto de novas colunas. Mas antes disso, defini uma função auxiliar para converter colunas para o tipo inteiro, tratando valores `NaN`. Devido a necessidade ao longo da resolução, tendo em vista a repetição destas 2 linhas de código.

In [8]:
def nan_to_zero_and_int(df, column_name):
    df[column_name] = df[column_name].fillna(0).astype(int)
    return df[column_name]

### 5.1. Contagem de Caracteres e Palavras

-   **`caracteres`**: Armazena o comprimento total de cada mensagem.
-   **`words`**: Armazena o número de palavras em cada mensagem.

In [17]:
file['caracteres'] = file['text_content_anonymous'].str.len()   
nan_to_zero_and_int(file, 'caracteres')

file['words'] = file['text_content_anonymous'].str.split().str.len()
nan_to_zero_and_int(file, 'words')

file[['text_content_anonymous', 'caracteres', 'words']].head()

,text_content_anonymous,caracteres,words
0,Então é Fato Renato o áudio que eu ouvi no wha...,110,20
1,"Saiu no YouTube do presidente a 8 horas atrás,...",141,23
2,"É isso, nossa parte já foi quase toda feita. N...",350,59
3,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,40,7
4,NaN,0,0


## 5. Engenharia de Atributos

Para enriquecer nossa análise, criamos novas colunas (features) a partir dos dados existentes. Primeiramente, definimos uma função auxiliar para tratar valores nulos em colunas numéricas.

In [18]:
def nan_to_zero_and_int(df, column_name):
    df[column_name] = df[column_name].fillna(0).astype(int)
    return df[column_name]

### 5.1. Contagem de Caracteres (`caracteres`)
Criamos a coluna `caracteres` para armazenar o comprimento total de cada mensagem. Essa métrica é útil para identificar mensagens anormalmente longas. Utilizamos o acessador vetorizado `.str.len()`, que é a forma mais performática para esta operação.

In [9]:
file['caracteres'] = file['text_content_anonymous'].str.len()   

nan_to_zero_and_int(file, 'caracteres')

file['caracteres'].head(15)

0      110
1      141
2      350
3       40
4        0
5      133
6     3669
7        0
8        0
9     1242
10    1257
11      98
12    1810
13    1257
14    1810
Name: caracteres, dtype: int64

### 5.2. Análise de Viralização e Compartilhamento

-   **`viral`**: Um marcador booleano (0 ou 1) que identifica se o texto de uma mensagem aparece em outras linhas.
-   **`sharings`**: Calcula a frequência exata de cada texto, utilizando `groupby()` e `transform('count')`.

In [10]:
file['viral'] = np.where(file.duplicated(subset=['text_content_anonymous'], keep=False), 1, 0)
file['sharings'] = file.groupby('text_content_anonymous')['text_content_anonymous'].transform('count')
nan_to_zero_and_int(file, 'sharings')

file[['text_content_anonymous', 'viral', 'sharings']].head()

,text_content_anonymous,viral,sharings
0,Então é Fato Renato o áudio que eu ouvi no wha...,0,1
1,"Saiu no YouTube do presidente a 8 horas atrás,...",0,1
2,"É isso, nossa parte já foi quase toda feita. N...",0,1
3,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,0,1
4,NaN,1,0


### 5.3. Análise de Sentimento

-   **`sentimental`**: Classifica o tom de cada mensagem como `-1` (negativo), `0` (neutro) ou `1` (positivo). Para isso, foi desenvolvido um classificador baseado em regras que utiliza léxicos e regras(heurísticas) para tratar modificadores de contexto (negações e intensificadores).

In [11]:
import re

LEXICO_POSITIVO = {
    'bom': 1, 'ótimo': 2, 'excelente': 2, 'maravilhoso': 2, 'corinthians': 1000, 'gostei': 1,
    'amo': 2, 'adoro': 2, 'incrível': 2, 'legal': 1, 'top': 1, 'recomendo': 2,
    'parabéns': 2, 'obrigado': 1, 'feliz': 2, 'sucesso': 2
}

LEXICO_NEGATIVO = {
    'ruim': -1, 'péssimo': -2, 'horrível': -2, 'detesto': -2, 'odeio': -2, 'disgrama': -2,
    'lixo': -2, 'merda': -2, 'bosta': -2, 'triste': -1, 'decepcionado': -2,
    'nunca': -1, 'jamais': -1, 'errado': -1, 'problema': -1, 'lento': -1
}

INTENSIFICADORES = ['muito', 'demais', 'extremamente', 'super']
NEGACOES = ['não', 'nem', 'nada']

def classificar_sentimento(texto):
    
    if not isinstance(texto, str):
        return 0

    texto = texto.lower()
    texto = re.sub(r'[^\w\s]', '', texto) 
    palavras = texto.split()
    
    score = 0

    for i, palavra in enumerate(palavras):
        
        modificador = 1 #neutro
        
        # Heurística: Verifica se a palavra ANTERIOR é um modificador
        if i > 0:
            palavra_anterior = palavras[i-1]
            if palavra_anterior in NEGACOES:
                modificador = -1 
            elif palavra_anterior in INTENSIFICADORES:
                modificador = 2 
        
        if palavra in LEXICO_POSITIVO:
            score += LEXICO_POSITIVO[palavra] * modificador
        elif palavra in LEXICO_NEGATIVO:
            score += LEXICO_NEGATIVO[palavra] * modificador

    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

file['sentimental'] = file['text_content_anonymous'].apply(classificar_sentimento)

file.loc[0:15, ['text_content_anonymous', 'sentimental']]

,text_content_anonymous,sentimental
0,Então é Fato Renato o áudio que eu ouvi no wha...,0
1,"Saiu no YouTube do presidente a 8 horas atrás,...",0
2,"É isso, nossa parte já foi quase toda feita. N...",1
3,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,0
4,NaN,0
5,Kķkkkkk to rindo até agora....Quem disse q ia ...,0
6,*SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...,-1
7,NaN,0
8,NaN,0
9,O Deputado Federal pelo NOVO e que foi candida...,0


## 6. Detecção e Remoção de "Trava-Zaps"

Conforme a tarefa (l), o dataset foi verificado em busca de mensagens maliciosas. A função `classificar_trava_zap` implementa um conjunto de heurísticas (comprimento extremo, caracteres especiais, etc.) para detectar e remover essas linhas.

In [13]:
def classificar_trava_zap(texto):
  
    if not isinstance(texto, str):
        return False

    if len(texto) > 3500:       #R1: msg muito longa
        return True

    caracteres_suspeitos = ['\u200c', '\u200d']     #R2: caracteres bugados (quebram renderizadores de texto)
    for char in caracteres_suspeitos:
        if char in texto:
            return True

    if len(texto) > 200:                                                          #R3: msg longa com poucos caracteres legíveis
        caracteres_legiveis = re.findall(r'[a-zA-Z0-9\s.,?!áéíóúâêôãõç]', texto)
        proporcao_legivel = len(caracteres_legiveis) / len(texto)
        if proporcao_legivel < 0.2:
            return True
            
    return False

file['eh_trava_zap'] = file['text_content_anonymous'].apply(classificar_trava_zap)

idx_to_drop = file[file['eh_trava_zap'] == True].index
print(f"Número de linhas ANTES da limpeza: {len(file)}")
print(f"Qtd de trava-zap encontrados: {len(idx_to_drop)}")

if not idx_to_drop.empty:
    file = file.drop(index=idx_to_drop)
else:
    print('Não há trava-zap no file.')

file.drop(columns=['eh_trava_zap'], inplace=True)



Número de linhas ANTES da limpeza: 557586
Qtd de trava-zap encontrados: 8237


## 7. Verificação de Inconsistências Lógicas

A etapa final do tratamento (tarefa m) é a verificação da consistência lógica entre os atributos. Para isso, defini algumas regras que o dataset deve seguir e implementei verificações para encontrar quaisquer linhas que violem essas condições.

In [14]:
problems_found = False

inconsist_midia = file[(file['has_media'] == True) & (file['media_type'].isna() | file['media_md5'].isna())]
if not inconsist_midia.empty:
    problems_found = True

inconsist_url = file[(file['has_media_url'] == True) & (file['media_url'].isna())]
if not inconsist_url.empty:
    problems_found = True

if not problems_found:
    print('Não foram encontradas inconsistências.')
else:
    print(f"Encontradas {len(inconsist_midia)} linhas onde 'has_media' é True, mas falta 'media_type' ou 'media_md5'.")
    print(inconsist_midia[['has_media', 'media_type', 'media_md5']].head())
    print(f"Encontradas {len(inconsist_url)} linhas onde 'has_media_url' é True, mas 'media_url' é nulo.")
    print(inconsist_url[['has_media_url', 'media_url']].head())

Não foram encontradas inconsistências.


### 7.1. Análise dos Resultados e Verificação por Amostragem

Após a execução do script, o resultado indicou que não foram encontradas inconsistências. Para aumentar a confiança neste resultado, realizei uma verificação final inspecionando uma amostra aleatória dos dados.

In [15]:
amostra = file.sample(n=50, random_state=42)

''' 
    Algumas possíveis amostras:

        amostra2 = file.sample(n=50, random_state=13)
        amostra3 = file.sample(n=50, random_state=69)

'''

amostra[['has_media', 'media_type', 'media_md5', 'has_media_url', 'media_url']].head(50)

,has_media,media_type,media_md5,has_media_url,media_url
313949,False,NaN,NaN,False,NaN
251961,False,NaN,NaN,False,NaN
101913,True,image/jpg,ff25865389c996d5d61ea5ccc6cb5422,False,NaN
297949,True,url,4d0a74f87f8fc7ddbec12f1bc3eadc9e,True,https://www1.folha.uol.com.br/blogs/hashtag/20...
40996,True,url,0bfe6617aab9f82356a213578dd5a63c,True,https://youtu.be/4nNQLRCr9XA
99421,True,url,688d2fdc5efd879d85197ee9f4dace08,True,https://gettr.com/post/p1uu8nade95
106966,True,image/jpg,cb0d0ffbf15314ff9abcd57d2e27fbe6,True,https://t.me/semcensuraoficial1
112988,False,NaN,NaN,True,https://youtu.be/7rwzgcu4Rqs
212598,True,url,27c333a48439838dfdc9c233cb792460,True,https://clouthub.com/v/1005eb42-4dfe-44fd-8cea...
162462,True,image/jpg,88b494cbc1d1305746d8dc6593c8c7bc,False,NaN
